# Work with Cosmos DB in Microsoft Fabric using the Cosmos DB Spark Connector

This notebook demonstrates how to use Spark and the Azure Cosmos DB Spark connector to read, query, analyze, and write data directly to an Azure Cosmos DB for NoSQL account in Microsoft Fabric. The Spark connector connects directly to the Cosmos DB endpoint to perform operations, which is different from using Spark to read mirrored data stored in OneLake.

## 📋 Prerequisites

Before starting, you'll need:
- An existing Cosmos DB database in Microsoft Fabric
- An existing container with sample data
- A Fabric notebook with Spark (Scala) kernel selected
- Custom Spark environment with Cosmos DB Spark Connector libraries (configured in Step 1)

> **Note**: This tutorial uses the built-in Cosmos DB sample created with a database name of **CosmosSampleDatabase** and a container name of **SampleData**.

## 🎯 Learning Objectives

- **Configure Spark Environment** - Set up custom libraries for Cosmos DB Spark Connector
- **Connect to Cosmos DB** - Authenticate and configure connection settings
- **Query Data with DataFrames** - Load and filter data using Spark operations
- **Use SparkSQL** - Query Cosmos DB using SQL syntax with Catalog API
- **Analyze Data** - Work with nested arrays and perform aggregations
- **Write Data** - Create new containers and write processed results

## 📚 What You'll Build

This notebook will guide you through:
1. Setting up your Spark environment with required libraries
2. Retrieving your Cosmos DB endpoint
3. Connecting to Cosmos DB using Spark connector
4. Querying and filtering data with DataFrames
5. Using SparkSQL for advanced queries
6. Analyzing price history data with array operations
7. Creating a new container and writing results

## ⚠️ IMPORTANT: Configuration Required

**Before running this notebook:**
1. Complete the Spark environment setup (see Step 1 below)
2. Get your Cosmos DB endpoint from the Fabric portal
3. Update the configuration values in the first code cell
4. Select **Spark (Scala)** as your notebook kernel
5. Attach your custom environment to this notebook

---

## Step 1: Configure Your Spark Environment

### Download Required Libraries

Download the latest Cosmos DB Spark Connector library files from Maven repository (Spark 3.5):

1. **azure-cosmos-spark_3-5_2-12-4.41.0.jar**
   - https://repo1.maven.org/maven2/com/azure/cosmos/spark/azure-cosmos-spark_3-5_2-12/4.41.0/azure-cosmos-spark_3-5_2-12-4.41.0.jar

2. **fabric-cosmos-spark-auth_3-1.1.0.jar**
   - https://repo1.maven.org/maven2/com/azure/cosmos/spark/fabric-cosmos-spark-auth_3/1.1.0/fabric-cosmos-spark-auth_3-1.1.0.jar

### Create Custom Spark Environment

1. **Create a new notebook** and select **Spark (Scala)** as the language
2. **Check workspace settings** to ensure you're using **Runtime 1.3 (Spark 3.5)**
3. Click the **environment dropdown** → **New environment**
4. **Provide an environment name** (e.g., "CosmosDB-Spark-Environment")
5. **Ensure Runtime 1.3 (Spark 3.5)** is selected
6. Choose **Custom Library** from the **Libraries** folder in the left panel
7. **Upload the two .jar files** you downloaded
8. Click **Save** → **Publish** → **Publish all** → **Publish**
9. **Verify** custom libraries show a status of **success**
10. **Return to this notebook** and attach the newly created environment:
    - Click environment dropdown → **Change environment** → Select your environment

---

## Step 2: Retrieve Your Cosmos DB Endpoint

### Get Endpoint from Fabric Portal

1. Open the **Fabric portal** (https://app.fabric.microsoft.com)
2. Navigate to your **existing Cosmos DB database**
3. Select the **Settings** option in the menu bar
4. Navigate to the **Connection** section
5. **Copy the value** of the "Endpoint for Cosmos DB NoSQL database" field
6. **Update the ENDPOINT variable** in the cell below with your copied value

---

In [ ]:
// Configure Cosmos DB connection settings
// UPDATE THESE VALUES with your Cosmos DB information

// User values for Cosmos DB
val ENDPOINT = "https://YourAccountEndpoint....cosmos.fabric.microsoft.com:443/"
val DATABASE = "CosmosSampleDatabase"  // Your Cosmos DB artifact name
val CONTAINER = "SampleData"  // Your container name

// Set configuration settings
val config = Map(
      "spark.cosmos.accountendpoint" -> ENDPOINT,
      "spark.cosmos.database" -> DATABASE,
      "spark.cosmos.container" -> CONTAINER,
      // auth config options
      "spark.cosmos.accountDataResolverServiceName" -> "com.azure.cosmos.spark.fabric.FabricAccountDataResolver",
      "spark.cosmos.auth.type" -> "AccessToken",
      "spark.cosmos.useGatewayMode" -> "true",
      "spark.cosmos.auth.aad.audience" -> "https://cosmos.azure.com/"
)

println("✅ Configuration loaded successfully!")
println(s"📍 Endpoint: $ENDPOINT")
println(s"🗄️  Database: $DATABASE")
println(s"📦 Container: $CONTAINER")

---

## Step 3: Query Data from Container Using DataFrames

Load OLTP data into a DataFrame to perform basic Spark operations. The Spark connector will infer the schema by sampling existing items in the container.

In [ ]:
// Read Cosmos DB container into a DataFrame
val df = spark.read.format("cosmos.oltp")
  .options(config)
  .option("spark.cosmos.read.inferSchema.enabled", "true")
  .load()

println("✅ Data loaded into DataFrame!")
println("📊 Showing first 5 rows:")

// Show the first 5 rows of the dataframe
df.show(5)

### Understanding the Schema

> **Note**: The *SampleData* container contains two different entity types with separate schemas:
> - **product** - Contains product information with pricing and inventory
> - **review** - Contains customer reviews with ratings
>
> The `inferSchema` option will detect both schemas within this Cosmos DB container.

In [ ]:
// Render schema to understand the data structure
println("📋 DataFrame Schema:")
df.printSchema()

---

## Step 4: Filter Data by Document Type

The container has two schemas distinguished by the `docType` property. Let's filter the data to work with each type separately.

In [ ]:
// Filter for products only using the where function
val productsDF = df.where("docType = 'product'")

println("🛍️  Showing product documents:")
productsDF.show(10)

In [ ]:
// Filter products by specific category using the filter function
val filteredDF = df
  .where("docType = 'product'")
  .filter($"categoryName" === "Computers, Laptops")

println("💻 Showing laptop products:")
filteredDF.show(10)

---

## Step 5: Query Cosmos DB Using SparkSQL

Configure the Catalog API to reference and manage Cosmos DB resources using Spark SQL queries.

In [ ]:
// Configure the Catalog API for SparkSQL access
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", config("spark.cosmos.accountendpoint"))
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.auth.type", "AccessToken")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.useGatewayMode", "true")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountDataResolverServiceName", "com.azure.cosmos.spark.fabric.FabricAccountDataResolver")

println("✅ Catalog API configured successfully!")

In [ ]:
// Query data using SparkSQL
val queryString = s"SELECT * FROM cosmosCatalog.$DATABASE.$CONTAINER"
val queryDF = spark.sql(queryString)

println("📊 Query results (first row):")
queryDF.show(1)

---

## Step 6: Analyze Price History with Array Operations

This example demonstrates how to work with embedded arrays in JSON documents. We'll:
1. Query the container for product data
2. Use the `explode` operator to flatten the priceHistory array
3. Calculate the lowest price for each product across its history

In [ ]:
// Retrieve product data with price history
val productPriceMinDF = spark.sql(
  "SELECT " +
  "  productId, " +
  "  categoryName, " +
  "  name, " +
  "  currentPrice, " +
  "  priceHistory " +
  "FROM cosmosCatalog." + DATABASE + "." + CONTAINER + " " +
  "WHERE " + CONTAINER + ".docType = 'product'"
)

// Prepare an exploded result set containing one row for every priceHistory entry
val explodedDF = productPriceMinDF
   .withColumn("priceHistory", explode(col("priceHistory")))
   .withColumn("priceDate", col("priceHistory").getField("date"))
   .withColumn("newPrice", col("priceHistory").getField("price"))

// Aggregate the lowest price ever recorded in the priceHistory
val lowestPriceDF = explodedDF
   .filter(col("docType") === "product")
   .groupBy("productId", "categoryName", "name")
   .agg(min("newPrice").as("lowestPrice"))

println("💰 Lowest historical prices by product:")
lowestPriceDF.show(10)

---

## Step 7: Create New Container and Write Data

Now we'll create a new container to store our processed results and write the lowest price data.

In [ ]:
// Create a new container named MinPricePerProduct using the Catalog API
val NEW_CONTAINER = "MinPricePerProduct"

val sqlDef = s"""
   |CREATE TABLE IF NOT EXISTS cosmosCatalog.$DATABASE.$NEW_CONTAINER 
   |USING cosmos.oltp 
   |TBLPROPERTIES(partitionKeyPath = '/id', autoScaleMaxThroughput = '1000')
""".stripMargin

spark.sql(sqlDef)

println(s"✅ Container '$NEW_CONTAINER' created successfully!")
println("📋 Container settings:")
println("   - Partition key: /id")
println("   - Autoscale throughput: 1000 RU/s")

In [ ]:
// All documents in Cosmos DB require an 'id' property
// Create an id column with the value of productId
val ProductsDF = lowestPriceDF.withColumn("id", col("productId"))

println("📊 Prepared data for writing (with id column):")
ProductsDF.show(10)

In [ ]:
// Configure the Cosmos DB connection for writing to the new container
val configWrite = Map(
   "spark.cosmos.accountendpoint" -> ENDPOINT,
   "spark.cosmos.database" -> DATABASE,
   "spark.cosmos.container" -> NEW_CONTAINER,
   "spark.cosmos.write.strategy" -> "ItemOverwrite",
   // auth config options
   "spark.cosmos.accountDataResolverServiceName" -> "com.azure.cosmos.spark.fabric.FabricAccountDataResolver",
   "spark.cosmos.auth.type" -> "AccessToken",
   "spark.cosmos.useGatewayMode" -> "true",
   "spark.cosmos.auth.aad.audience" -> "https://cosmos.azure.com/"
)

println("✅ Write configuration created!")

In [ ]:
// Write the DataFrame to the new container
ProductsDF.write
  .format("cosmos.oltp")
  .options(configWrite)
  .mode("APPEND")
  .save()

println("✅ Data written successfully to MinPricePerProduct container!")

---

## Step 8: Verify the Write Operation

Query the new container to validate that it contains the correct data.

In [ ]:
// Test our write operation worked
val queryString = s"SELECT * FROM cosmosCatalog.$DATABASE.$NEW_CONTAINER"
val queryDF = spark.sql(queryString)

println("📊 Verifying data in new container:")
queryDF.show(10)

---

## 🎉 Summary

### What You've Accomplished

✅ **Configured Spark Environment** - Set up custom libraries for Cosmos DB Spark Connector  
✅ **Connected to Cosmos DB** - Authenticated using Fabric-specific configuration  
✅ **Queried with DataFrames** - Loaded and filtered data using Spark operations  
✅ **Used SparkSQL** - Queried Cosmos DB using SQL syntax with Catalog API  
✅ **Analyzed Arrays** - Worked with nested priceHistory data using explode  
✅ **Performed Aggregations** - Calculated minimum prices across product history  
✅ **Created Containers** - Used Catalog API to create new Cosmos DB containers  
✅ **Wrote Data** - Persisted processed results back to Cosmos DB  

### Key Concepts Demonstrated

- **Direct Cosmos DB Access** - Using Spark connector vs. mirrored OneLake data
- **Schema Inference** - Automatic schema detection for multiple document types
- **DataFrame Operations** - Filtering, transformations, and aggregations
- **Array Operations** - Using `explode` to work with nested arrays
- **Catalog API** - Managing Cosmos DB resources through SparkSQL
- **Write Strategies** - Using ItemOverwrite to handle upsert operations

### Next Steps

- Explore additional **DataFrame transformations** on your data
- Implement **batch processing** patterns for large datasets
- Create **analytical queries** using window functions
- Build **data pipelines** combining multiple containers
- Integrate with **Power BI** for visualization

### 📚 Additional Resources

- [Azure Cosmos DB Spark Connector Documentation](https://learn.microsoft.com/azure/cosmos-db/nosql/spark-connector)
- [Microsoft Fabric Spark Documentation](https://learn.microsoft.com/fabric/data-engineering/)
- [Cosmos DB in Fabric Overview](https://learn.microsoft.com/fabric/database/cosmos-db/)

---

**Happy querying with Cosmos DB and Spark! 🚀**